# Random Perturbations

Streamlined notebook for evaluating random perturbations on saved models and datasets.

## Imports

In [1]:
# Standard library
import os
import sys
import time

# Third-party
import matplotlib.pyplot as plt
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim

# Local package imports
from minima_volume.perturb_funcs import ( analyze_wiggles_metrics )

from minima_volume.dataset_funcs import (
    load_dataset,
    load_model,
    load_models_and_data,
    prepare_datasets,
    tensor_to_list,
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Input Parameters

In [2]:
# Perturbation Configuration
perturbation_seed = 1
num_directions = 1000
N = 100
x = np.linspace(0, 1, N)
coefficients = x**2

# Other Configuration
dataset_quantities = [0, (int)(0.09*9409), (int)(0.19*9409), (int)(0.39*9409), (int)(0.59*9409)] # modulo arithmetic dataset size
base_output_dir = ""

## Model + Dataset Specific Code

This is for model and dataset specific code.

In [3]:
# User specifies the model module name
from minima_volume.models import modulo_arithmetic_model_data as model_module
modulus = 97

# MNIST specific initialization parameters
hidden_dims = [250]

# Grab model
model_template = model_module.get_model(N = modulus, hidden_dims=hidden_dims, device=device, seed=0)

# Grab loss and metrics
loss_fn = model_module.get_loss_fn()
other_metrics = model_module.get_additional_metrics()

## Loading Model and Datasets

In [4]:
# ====================================
# Load Trained Models and Dataset
# ====================================
target_dir = "models_and_data"  # relative path
loaded_models, loaded_model_data, loaded_dataset = load_models_and_data(
    model_template=model_template,
    target_dir=target_dir,
    device=device,
)

# Dataset Info
dataset_type = loaded_dataset['dataset_type']
print(f"Dataset type: {dataset_type}")
print(f"Dataset quantities: {loaded_dataset['dataset_quantities']}")

print("\nTensor shapes:")
for key in ["x_base_train", "y_base_train", "x_additional", "y_additional", "x_test", "y_test"]:
    shape = getattr(loaded_dataset[key], "shape", None)
    print(f"  {key}: {shape if shape is not None else 'None'}")

# Reconstruct trained_model dicts safely.
# If the loss or accuracy or additional metrics happen to be
# tensors, they get safely converted to lists.
all_models = [
    {
        "model": model,
        **{
            k: tensor_to_list(model_data[k], key_path=k)
            for k in ["train_loss", "train_accs", "test_loss", "test_accs", "additional_data", "dataset_type"]
        },
    }
    for model, model_data in zip(loaded_models, loaded_model_data)
]
print(f"Reconstructed {len(all_models)} trained models")

Looking for models and dataset in: models_and_data
Found 5 model files:
  - model_additional_0.pt
  - model_additional_1787.pt
  - model_additional_3669.pt
  - model_additional_5551.pt
  - model_additional_846.pt
✅ Model loaded into provided instance from models_and_data\model_additional_0.pt
Successfully loaded: model_additional_0.pt
✅ Model loaded into provided instance from models_and_data\model_additional_1787.pt
Successfully loaded: model_additional_1787.pt
✅ Model loaded into provided instance from models_and_data\model_additional_3669.pt
Successfully loaded: model_additional_3669.pt
✅ Model loaded into provided instance from models_and_data\model_additional_5551.pt
Successfully loaded: model_additional_5551.pt
✅ Model loaded into provided instance from models_and_data\model_additional_846.pt
Successfully loaded: model_additional_846.pt

Model data loaded from all models:
Model 0 (model_additional_0.pt):
  - Additional data: 0
  - Dataset type: data
  - Training accuracies: 500 e

## Perturbations

Using the saved datasets, we perform model perturbations. 

In [5]:
device = 'cuda'

x_base_train = loaded_dataset['x_base_train'].to(device)
y_base_train = loaded_dataset['y_base_train'].to(device)
x_additional = loaded_dataset['x_additional'].to(device)
y_additional = loaded_dataset['y_additional'].to(device)
x_test = loaded_dataset['x_test'].to(device)
y_test = loaded_dataset['y_test'].to(device)

# Loss function and metrics already grabbed from the model module
analyze_wiggles_metrics(
    model_list = all_models, 
    x_base_train = x_base_train,
    y_base_train = y_base_train, 
    x_additional = x_additional,
    y_additional = y_additional,
    x_test = x_test,
    y_test = y_test, 
    dataset_quantities = dataset_quantities, 
    dataset_type = dataset_type, 
    metrics = {"loss": loss_fn, **other_metrics}, 
    coefficients = coefficients,
    num_directions = num_directions,
    perturbation_seed = perturbation_seed,
    base_output_dir = base_output_dir,
    device = device,  # can be set to GPU if needed
)


""" Our saved results are structured as follows:
wiggle_results: List of dictionaries containing wiggle test results
Each dictionary is of the form
{
'loss':
'coefficients':
'accs':
'perturbation_seed':
'perturbation_norm':
}
model: PyTorch model used in analysis (state_dict will be saved)
output_dir: Directory to save results (default: "imgs/swiss/random_dirs")
filename: Name of output file (default: "random_directions.npz")
**kwargs: Additional key-value pairs to be saved in the output file
Typically:
'additional_data':
'model_trained_data':
'dataset_type':
'base_dataset_size': 
'test_loss':
'test_accs':
'num_params':
"""

The number of parameters of the perturbation is 73097
Testing on data with 0 samples - 1000 directions
Testing model trained on 0 additional data.
Loss: 0.0155
Accs: 0.0116


Wiggle completed in 93.86 seconds for data model trained with 0 samples
Saved to data_0

Testing model trained on 1787 additional data.
Loss: 0.0155
Accs: 0.2028


Wiggle completed in 91.00 seconds for data model trained with 1787 samples
Saved to data_0

Testing model trained on 3669 additional data.
Loss: 0.0088
Accs: 0.7491


Wiggle completed in 92.77 seconds for data model trained with 3669 samples
Saved to data_0

Testing model trained on 5551 additional data.
Loss: 0.0028
Accs: 1.0000


Wiggle completed in 92.90 seconds for data model trained with 5551 samples
Saved to data_0

Testing model trained on 846 additional data.
Loss: 0.0223
Accs: 0.1027


...

Wiggle completed in 92.81 seconds for data model trained with 5551 samples
Saved to data_1787

Testing model trained on 846 additional data.
Testing on data with 3669 samples - 1000 directions
Testing model trained on 0 additional data.
Testing model trained on 1787 additional data.
Testing model trained on 3669 additional data.
Loss: 0.0088
Accs: 0.7491


Wiggle completed in 93.40 seconds for data model trained with 3669 samples
Saved to data_3669

Testing model trained on 5551 additional data.
Loss: 0.0028
Accs: 1.0000


Wiggle completed in 93.27 seconds for data model trained with 5551 samples
Saved to data_3669

Testing model trained on 846 additional data.
Testing on data with 5551 samples - 1000 directions
Testing model trained on 0 additional data.
Testing model trained on 1787 additional data.
Testing model trained on 3669 additional data.
Testing model trained on 5551 additional data.
Loss: 0.0028
Accs: 1.0000


Wiggle completed in 91.01 seconds for data model trained with 5551 samples
Saved to data_5551

Testing model trained on 846 additional data.


' Our saved results are structured as follows:\nwiggle_results: List of dictionaries containing wiggle test results\nEach dictionary is of the form\n{\n\'loss\':\n\'coefficients\':\n\'accs\':\n\'perturbation_seed\':\n\'perturbation_norm\':\n}\nmodel: PyTorch model used in analysis (state_dict will be saved)\noutput_dir: Directory to save results (default: "imgs/swiss/random_dirs")\nfilename: Name of output file (default: "random_directions.npz")\n**kwargs: Additional key-value pairs to be saved in the output file\nTypically:\n\'additional_data\':\n\'model_trained_data\':\n\'dataset_type\':\n\'base_dataset_size\': \n\'test_loss\':\n\'test_accs\':\n\'num_params\':\n'